In [ ]:
import numpy as np
from scipy.signal import convolve2d


def to_numpy(input):
    """convert matrix of chars to numpy array of ints"""
    table = []
    for line in input.strip().split("\n"):
        row = [ord(c) for c in line.strip()]
        table.append(row)

    return np.array(table)


input_small = """AAAA
BBCD
BBCC
EEEC"""

input_medium = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

with open("input.txt") as f:
    input_large = f.read()

to_numpy(input_large)

array([[73, 73, 73, ..., 87, 87, 87],
       [73, 73, 65, ..., 87, 88, 87],
       [73, 73, 65, ..., 88, 88, 87],
       ...,
       [84, 84, 87, ..., 72, 72, 72],
       [84, 87, 87, ..., 72, 72, 72],
       [84, 84, 84, ..., 72, 72, 72]])

In [6]:
def check_is_outside(matrix, x, y):
    return x < 0 or y < 0 or x >= matrix.shape[0] or y >= matrix.shape[1]


def explore_region(start_point, matrix, banned_points=None):
    value = matrix[start_point]
    if banned_points is None:
        banned_points = set()
    visited_region_points = {start_point}
    stack = [start_point]
    perimeter = 0
    while stack:
        x, y = stack.pop()
        adjacent_points = {(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)}
        adjacent_points = {
            point
            for point in adjacent_points
            if (not check_is_outside(matrix, *point) and matrix[point] == value)
        }
        n_edges = 4 - len(adjacent_points)
        perimeter += n_edges
        for point in adjacent_points:
            if point not in visited_region_points and point not in banned_points:
                stack.append(point)
                visited_region_points.add(point)
    return visited_region_points, value, perimeter


def get_price(matrix):
    all_points = set(
        (x, y) for x in range(matrix.shape[0]) for y in range(matrix.shape[1])
    )
    regions = []
    banned_points = set()
    total_price = 0
    while all_points:
        start_point = all_points.pop()
        region, val, perimeter = explore_region(start_point, matrix, banned_points)
        regions.append(region)
        banned_points.update(region)
        all_points -= region
        area = len(region)
        price = area * perimeter
        total_price += price

    return total_price


get_price(to_numpy(input_large))

1375574

## Second part

In [7]:
def count_consecutive_regions(vector):
    n_regions = 0
    for i in range(1, len(vector)):
        if vector[i] != vector[i - 1] and vector[i - 1] != 0:
            n_regions += 1
    if vector[-1] != 0:
        n_regions += 1
    return n_regions


assert count_consecutive_regions([1, 1, 1, 0, 0, 1, 1]) == 2
assert count_consecutive_regions([1, 1, 1, 0, 0, 0, 1]) == 2
assert count_consecutive_regions([1, 1, 1, 0, 0, 0, 0]) == 1
assert count_consecutive_regions([0, 0, 0, 1, 1, 1, 1]) == 1
assert count_consecutive_regions([0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1]) == 4
assert count_consecutive_regions([0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 3]) == 5


def get_n_sides(region):
    min_x = min(x for x, y in region)
    min_y = min(y for x, y in region)
    max_x = max(x for x, y in region)
    max_y = max(y for x, y in region)
    matrix = np.zeros((max_x + 1 - min_x, max_y + 1 - min_y), dtype=bool)

    for x, y in region:
        matrix[x - min_x, y - min_y] = True
    # conv2d to detect edges
    h_kernel = np.array([[1, -1]])
    v_kernel = np.array(
        [
            [1],
            [
                -1,
            ],
        ]
    )
    # print(matrix)
    h_edges = convolve2d(matrix, h_kernel, mode="full")
    n_vertical_lines = 0
    for col in h_edges.T:
        n_vertical_lines += count_consecutive_regions(col)

    # print(h_edges)
    # print(h_edges.cumsum(axis=1))
    v_edges = convolve2d(matrix, v_kernel, mode="full")
    n_horizontal_lines = 0
    for row in v_edges:
        n_horizontal_lines += count_consecutive_regions(row)
    # print(v_edges)

    return n_vertical_lines + n_horizontal_lines


In [39]:
def get_price2(matrix):
    all_points = set(
        (x, y) for x in range(matrix.shape[0]) for y in range(matrix.shape[1])
    )
    regions = []
    banned_points = set()
    total_price = 0
    while all_points:
        start_point = all_points.pop()
        region, val, perimeter = explore_region(start_point, matrix, banned_points)
        regions.append(region)
        banned_points.update(region)
        all_points -= region
        area = len(region)
        n_sides = get_n_sides(region)
        price = area * n_sides
        total_price += price
        # print(
        #     f"Region {chr(val)} has area {area} and {n_sides} sides, with price {price}"
        # )

    return total_price


input_ex = """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""

input_insides = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA"""

assert get_price2(to_numpy(input_large)) == 830566

## Other solution, not working yet

In [ ]:
def in_region(point, matrix, region_value):
    return not check_is_outside(matrix, *point) and matrix[point] == region_value


def corner_counter(center_point, matrix):
    x, y = center_point
    val = matrix[x, y]
    corners = 0
    diagonals = {
        "ul": (x - 1, y - 1),
        "ur": (x + 1, y - 1),
        "dl": (x - 1, y + 1),
        "dr": (x + 1, y + 1),
    }
    straights = {"u": (x, y - 1), "d": (x, y + 1), "l": (x - 1, y), "r": (x + 1, y)}
    for key in diagonals.keys():
        # normal corner. if ul: not u and not l
        # o -
        # - x
        cond1 = not in_region(straights[key[0]], matrix, val) and not in_region(
            straights[key[1]], matrix, val
        )
        # inside corner. if ul: u and l and not d (ul)
        # - x
        # x x
        cond2 = (
            in_region(straights[key[0]], matrix, val)
            and in_region(straights[key[1]], matrix, val)
            and not in_region(diagonals[key], matrix, val)
        )

        if cond1 or cond2:
            corners += 1
    return corners


# test
matrix = to_numpy(input_insides)
print(matrix)
center_point = (5, 0)

corner_counter(center_point, matrix)

[[65 65 65 65 65 65]
 [65 65 65 66 66 65]
 [65 65 65 66 66 65]
 [65 66 66 65 65 65]
 [65 66 66 65 65 65]
 [65 65 65 65 65 65]]


2

In [ ]:
def explore_region_with_corners(start_point, matrix, banned_points=None):
    value = matrix[start_point]
    if banned_points is None:
        banned_points = set()
    visited_region_points = {start_point}
    stack = [start_point]
    perimeter = 0
    n_corners = 0

    while stack:
        x, y = stack.pop()
        adjacent_points = {(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)}
        adjacent_points = {
            point
            for point in adjacent_points
            if (not check_is_outside(matrix, *point) and matrix[point] == value)
        }
        n_edges = 4 - len(adjacent_points)
        adjacent_points = list(adjacent_points)
        n_corners += corner_counter((x, y), matrix)

        perimeter += n_edges
        for point in adjacent_points:
            if point not in visited_region_points and point not in banned_points:
                stack.append(point)
                visited_region_points.add(point)
    return visited_region_points, value, n_corners


def get_price2b(matrix):
    all_points = set(
        (x, y) for x in range(matrix.shape[0]) for y in range(matrix.shape[1])
    )
    regions = []
    banned_points = set()
    total_price = 0
    while all_points:
        start_point = all_points.pop()
        region, val, n_corners = explore_region_with_corners(
            start_point, matrix, banned_points
        )
        regions.append(region)
        banned_points.update(region)
        all_points -= region
        area = len(region)
        price = area * n_corners
        total_price += price

    return total_price


assert get_price2b(to_numpy(input_large)) == 830566